# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

In [2]:
from datasets import load_dataset

ds = load_dataset('glue', 'mrpc')

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

En otras palabras, seleccionamos el subset `mrpc` del dataset `glue`:

In [3]:
ex = ds['train'][0]
print(ex)

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


In [6]:
# type of ex is dict
print(ds['train'])

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})


Así se ve un ejemplo. Notamos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [7]:
labels = ds['train'].features['label']

In [8]:
labels.int2str(0)

'not_equivalent'

### Tokenizando

¿Recuerdas que con visión descargamos el feature extractor directamente del repositorio del modelo pre-entrenado que vamos a usar como base?

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Descargamos el tokenizador directamente del repo del modelo que usaremos.

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador. 

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Por ejemplo, durante esta clase utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.

En la organización del Platzi en el Hub puedes encontrar un [modelo BERT](https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel) afinado siguiendo el mismo proceso que usamos en esta clase.

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño. 

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica 

### Configurando `Trainer`


### Entrenamiento

### Evaluación

### Compartimos en el Hub